# *ImageRearranger* Jupyter Notebook

*This notebook arranges a grid of images into a mosaic according to their similarity.*

![overview.png](overview.png)

[This notebook](https://github.com/golanlevin/ImageRearranger/blob/master/ImageRearranger.ipynb) (ImageRearranger.ipynb):

1. Loads a collection of images (either as a zipped file, a directory, or in the form of a single mosaic image).
2. Computes high-dimensional features which describe the images (either using an [image pyramid](https://en.wikipedia.org/wiki/Pyramid_(image_processing)), or a neural network).
3. [Reduces the dimensionality](https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction) of those features to a 2D point cloud (either using [UMAP](https://umap-learn.readthedocs.io/en/latest/) or [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding))
4. Rectifies the 2D point cloud into a grid (by solving the [Linear Assignment Problem](https://en.wikipedia.org/wiki/Assignment_problem)), using that grid to produce an ordered mosaic of the input images.
5. Saves the final mosaic image, as well as JSON files containing the 2D point cloud and grid locations.

A very similar [Google Colab version of this notebook is available here](https://colab.research.google.com/drive/1rgxYnSziGuToW0oLbmzSwq470e6nVXHC?usp=sharing).

---

### Instructions

* You will need Python 3.11 or newer. *This notebook was developed with Python 3.13 on MacOSX 14.5.*
  * First, **check** if Python3 is installed. Run: `python3 --version`
  * If you see output like `Python 3.x.x`, then Python is installed. If not, install it via Homebrew: `brew install python`
  * Then **verify**: `python3 --version`
* **Download** [this repository](https://github.com/golanlevin/ImageRearranger):
  * (**Option 1: Download as ZIP**) In GitHub, **click** on the green "Code" button in the upper right and **select** "Download ZIP" from the pulldown that appears). This will download a zip file to your computer. **Unzip** the compressed file. I recommend that you **rename** the folder to `ImageRearranger/`.
  * (**Option 2: Clone via Git**) In your terminal, **navigate** to where you want to save the repository, and run this command to clone it: `git clone https://github.com/golanlevin/ImageRearranger.git`
* At your Terminal prompt, **change directory** to that folder: `cd ImageRearranger`
* Because this notebook requires the installation of numerous Python libraries, *working in a virtual environment (venv) is extremely strongly recommended*. **Create** a virtual environment, e.g. `mosaicVenv`:
  * Mac: `python3 -m venv mosaicVenv`
  * Win: `python -m venv mosaicVenv`
* **Activate** the `mosaicVenv` virtual environment:
  * Mac: `source mosaicVenv/bin/activate`
  * Win: `mosaicVenv\Scripts\activate`
* Before we install necessary libraries, it's optional but recommended that you **upgrade** pip first: `pip install --upgrade pip`
* **Install** all required packages (making sure that `mosaicVenv` is activated first!): `pip install -r requirements.txt`.
  * This will install: `numpy scipy scikit-learn Pillow jupyter matplotlib opencv-python scikit-image umap-learn torch torchvision
git+https://github.com/gatagat/lap.git`
  * You won't need to re-install libraries the next time you activate the virtual environment.
* You should now be able to **launch** the [ImageRearranger.ipynb](ImageRearranger.ipynb) Jupyter Notebook (if you haven't already) with `jupyter notebook`
  * This will open `http://localhost:8888/tree`
  * From there, **open** `http://localhost:8888/notebooks/ImageRearranger.ipynb`
* **Step through** this notebook! You can step through this notebook using options in the Run menu. I recommend stepping through cell by cell with **Shift-Return**.

---

### Quickstart

This assumes you have already constructed the virtual environment, and installed the libraries, etc.:

* `cd ImageRearranger` 
* `source mosaicVenv/bin/activate`
* `jupyter notebook`

---

### Credits

* Based on Kyle McDonald's [ImageRearranger](https://github.com/kylemcdonald/ImageRearranger/tree/master?tab=readme-ov-file).
* Includes code from Kyle McDonald's [python-utils](https://github.com/kylemcdonald/python-utils) repository. 
* Inspired by [this collection](https://twitter.com/JUSTIN_CYR/status/829196024631681024) of pixel art by Justin Cyr.
* Demo includes tiles of the [Kress Collection](https://www.kressfoundation.org/kress-collection) at the US National Gallery of Art.
* Updates & Extensions by Golan Levin, February 2025


---
# Settings, Imports, and Definitions

In [ ]:
# ImageRearranger
# Arranges a grid of images into a mosaic according to their similarity.

# Practical limits on this tool: 
# - Optimizing a grid for more than ~5K points will start to become EXTREMELY slow. 
# - Because the cost matrix is n^2 in memory, ~45K+ points exceed a 32GB RAM ceiling.

# -----------------------------------------------
# GLOBAL SETTINGS
# This program loads a pre-made mosaic image or a folder of square images.

LOAD_FROM_FOLDER = True  # ✅ Set to True to load tiles from a folder instead of a mosaic.
LOAD_FOLDER_FROM_ZIP = False # ✅ Set to True to extract folder from a zip file. 

IMAGE_FILENAME = "inputs/src_cyr_32.png"  # ✅ Used if LOAD_FROM_FOLDER = False
IMAGE_FOLDER = "inputs/nga_kress_32"      # ✅ Used if LOAD_FROM_FOLDER = True
IMAGE_FOLDER_ZIPFILE = "inputs/nga_kress_32.zip" # ✅ Used if LOAD_FROM_FOLDER = True
TILE_SIZE = 32                            # ✅ Define tile size for image processing

# -----------------------------------------------
# Feature extraction method
# Choose between "pyramid" (multi-scale image blurring) or "cnn" (deep learning features)
FEATURE_METHOD = "cnn"  # ✅ Options: "pyramid" or "cnn"

# Dimensionality reduction method
REDUCTION_METHOD = "TSNE" # ✅ Options: "UMAP" or "TSNE"


In [ ]:
# -------------------------------
# NOTEBOOK SETUP

# Enable inline plotting for Jupyter notebooks
%matplotlib inline

# Import necessary libraries
import os
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import IPython
from umap import UMAP

# Improve matplotlib rendering quality for retina displays
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

# Improve image rendering in Jupyter
IPython.core.display.display_html(
    IPython.core.display.HTML("<style>img{image-rendering: pixelated}</style>")
)

In [ ]:
# -------------------------------
# IMAGE UTILITY FUNCTIONS

# Reads an image from a file using OpenCV.
# Returns the loaded image as a NumPy array.
def imread(filename, mode=None):
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Image file '{filename}' not found.")
    if mode == 'rgb':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif mode == 'gray':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


# Displays an image (a NumPy array) in a Jupyter notebook using Matplotlib.
# Increases display resolution if `retina` is True.
def imshow(img, retina=False):
    if img is None:
        raise ValueError("Input image is None.")
    plt.figure(figsize=(6, 6) if retina else (4, 4))
    cmap = 'gray' if len(img.shape) == 2 else None
    plt.imshow(img, cmap=cmap)
    plt.axis('off')
    plt.show()


# Loads all images from a folder; uses the `imread()` function above.
def load_images_from_folder(folder_path, target_size=32, debug=True):
    """
    Parameters:
        folder_path (str): Path to the folder containing images.
        target_size (int): Target size (assumes square images, default=32).
        debug (bool): If True, prints debug information.
    Returns:
        np.ndarray: Array of images in (num_images, 32, 32, 3) format.
        list: List of successfully loaded image filenames (sorted order).
    """
    images = []
    filenames = []
    
    # Get all valid image files (JPG and PNG), sorted numerically
    valid_extensions = {".jpg", ".jpeg", ".png"}
    files = sorted(
        [f for f in os.listdir(folder_path) 
         if os.path.splitext(f.lower())[1] in valid_extensions]
    )

    if debug:
        print(f"Found {len(files)} potential images in '{folder_path}':")
        print(files[:10])  # Print first 10 filenames (to check sorting)

    for filename in files:
        file_path = os.path.join(folder_path, filename)
        try:
            # Use the existing `imread()` function
            img = imread(file_path, mode='rgb')

            # Ensure image is the target size (resize if necessary)
            if img.shape[:2] != (target_size, target_size):
                img = cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_LANCZOS4)
            
            # Store image and filename
            images.append(img)
            filenames.append(filename)

        except FileNotFoundError:
            if debug:
                print(f"❌ Skipping missing file: {filename}")
            continue  # Skip missing/corrupt files
        except Exception as e:
            if debug:
                print(f"⚠️ Error loading {filename}: {e}")
            continue  # Skip problematic files
    if debug:
        print(f"✅ Successfully loaded {len(images)} images.")
    return np.array(images), filenames


# -------------------------------
# IMAGE MOSAIC FUNCTIONS

# Swap axes of an array (supports NumPy and PyTorch).
def swapaxes(x, a, b):
    try:
        return x.swapaxes(a, b)
    except AttributeError:  # Support PyTorch tensors
        return x.transpose(a, b)


# Arranges a batch of images into a mosaic grid.
def make_mosaic(x, nx=None, ny=None):
    """
    Parameters:
        x (np.ndarray): Image batch of shape (n, h, w) or (n, h, w, c).
        nx (int): Number of columns in the mosaic.
        ny (int): Number of rows in the mosaic.
    Returns:
        np.ndarray: Mosaic image.
    """
    if not isinstance(x, np.ndarray):
        x = np.asarray(x)

    n, h, w = x.shape[:3]
    has_channels = len(x.shape) > 3
    c = x.shape[3] if has_channels else None

    if nx is None and ny is None:
        ny = int(np.sqrt(n))  # Default to a roughly square layout
        nx = (n + ny - 1) // ny  # Ensure enough columns
    elif ny is None:
        ny = n // nx
    elif nx is None:
        nx = n // ny

    end_shape = (w, c) if has_channels else (w,)
    mosaic = x.reshape(ny, nx, h, *end_shape)
    mosaic = swapaxes(mosaic, 1, 2)
    mosaic = mosaic.reshape(ny * h, nx * w, c) if has_channels else mosaic.reshape(ny * h, nx * w)

    return mosaic


# Splits a mosaic image into individual images.
# Note: `//` is the Python floored-division operator. 
def unmake_mosaic(mosaic, nx=None, ny=None, w=None, h=None):
    """
    Parameters:
        mosaic (np.ndarray): The mosaic image.
        nx (int): Number of images per row.
        ny (int): Number of images per column.
        w (int): Width of each sub-image.
        h (int): Height of each sub-image.
    Returns:
        np.ndarray: Array of split images.
    """
    hh, ww = mosaic.shape[:2]

    if nx is not None or ny is not None:
        if nx is None:
            h = hh // ny
            w = h
            nx = ww // w
        elif ny is None:
            w = ww // nx
            h = w
            ny = hh // h
        else:
            w = ww // nx
            h = hh // ny
    elif w is not None or h is not None:
        if w is None:
            w = h
        elif h is None:
            h = w
        nx = ww // w
        ny = hh // h

    end_shape = (w, mosaic.shape[2]) if len(mosaic.shape) > 2 else (w,)

    x = mosaic.reshape(ny, h, nx, *end_shape)
    x = swapaxes(x, 1, 2)
    x = x.reshape(-1, h, *end_shape)

    return x


# -------------------------------
# IMAGE PLOTTING FUNCTIONS

# Places images at given (x, y) coordinates onto a blank canvas.
def plot_images(images, xy, blend=np.maximum, canvas_shape=(512, 512), fill=0):
    """
    Parameters:
        images (np.ndarray): Array of images to place.
        xy (np.ndarray): (x, y) positions for each image.
        blend (func): Blending function for overlapping images (default: np.maximum).
        canvas_shape (tuple): Shape of output canvas (height, width, channels).
        fill (int): Fill value for empty pixels (default: 0).
    Returns:
        np.ndarray: Final composed image.
    """
    h, w = images.shape[1:3]
    if images.ndim == 4:
        canvas_shape = (canvas_shape[0], canvas_shape[1], images.shape[3])

    min_xy = np.amin(xy, 0)
    max_xy = np.amax(xy, 0)

    min_canvas = np.array((0, 0))
    max_canvas = np.array((canvas_shape[0] - h, canvas_shape[1] - w))

    xy_mapped = min_canvas + (xy - min_xy) * (max_canvas - min_canvas) / (max_xy - min_xy)
    xy_mapped = xy_mapped.astype(int)

    canvas = np.full(canvas_shape, fill)
    for image, pos in zip(images, xy_mapped):
        x_off, y_off = pos
        sub_canvas = canvas[y_off:y_off+h, x_off:x_off+w]
        sub_image = image[:h, :w]
        try:
            canvas[y_off:y_off+h, x_off:x_off+w] = blend(sub_canvas, sub_image)
        except ValueError:
            print(pos, h, w, min_canvas, max_canvas)
            raise

    return canvas

# 1. Load a collection of images 
### Images can be loaded as a zipped collection, a directory, or as a single mosaic image.

In [ ]:
# -----------------------------------
# LOAD IMAGE DATA (MOSAIC OR FOLDER)

import zipfile
import shutil

if LOAD_FROM_FOLDER:

    if (LOAD_FOLDER_FROM_ZIP):
        # Check if the folder exists
        if not os.path.exists(IMAGE_FOLDER):
            print(f"📦 Extracting {IMAGE_FOLDER_ZIPFILE}...")
            with zipfile.ZipFile(IMAGE_FOLDER_ZIPFILE, 'r') as zip_ref:
                zip_ref.extractall("inputs")  # ✅ Extract into the same folder
            print(f"✅ Extracted images to {IMAGE_FOLDER}")
            # Delete the unwanted __MACOSX folder if it exists
            UNWANTED_MACOSX_PATH = "inputs/__MACOSX"
            if os.path.exists(UNWANTED_MACOSX_PATH):
                shutil.rmtree(UNWANTED_MACOSX_PATH)
    
    # Load images from the specified folder
    print(f"Loading images from folder: {IMAGE_FOLDER}")
    images, filenames = load_images_from_folder(IMAGE_FOLDER, TILE_SIZE, False)
    print(f"Loaded {len(images)} tiles from {IMAGE_FOLDER}")

    # Construct a mosaic from the loaded tiles for visualization
    aspectRatio = 1.618 # Golden Ratio
    nx = round(np.sqrt(aspectRatio * len(images)))  # Estimate layout
    ny = (len(images) + nx - 1) // nx  # Ensure enough rows

    # Compute the number of missing images for a complete grid
    required_images = nx * ny
    n_loaded_images = len(images)
    missing_tiles = required_images - n_loaded_images
    
    if missing_tiles > 0:
        print(f"⚠️ Missing {missing_tiles} tiles for complete grid; padding with black tiles.")
        black_tiles = np.zeros((missing_tiles, TILE_SIZE, TILE_SIZE, 3), dtype=np.uint8)  
        images = np.concatenate((images, black_tiles), axis=0)  # Concatenate missing tiles

    # Construct and display a "mosaic" image from the loaded tiles.
    img = make_mosaic(images, nx=nx, ny=ny)  
    print(f"Constructed mosaic from {len(images)} tiles.")
    imshow(img, retina=True)
    
else:
    # Load full mosaic image
    print(f"Loading mosaic image: {IMAGE_FILENAME}")
    img = imread(IMAGE_FILENAME, 'rgb')  
    imshow(img, retina=True)

    # Convert mosaic image into individual tiles
    # - `img` is the full input image (assumed to be a mosaic of tiles).
    # - `unmake_mosaic(img, w=TILE_SIZE)` extracts individual square tiles.
    images = unmake_mosaic(img, w=TILE_SIZE)
    n_loaded_images = len(images)  # Should be `nx * ny`
    filenames = [f"tile_{i:04d}" for i in range(len(images))]


# Compute number of columns (nx) and rows (ny) in the mosaic
nx = img.shape[1] // TILE_SIZE  # Number of tiles per row
ny = img.shape[0] // TILE_SIZE  # Number of tiles per column

# Print the total number of extracted images
print(f"✅ Processing {len(images)} images, each of size {TILE_SIZE}×{TILE_SIZE} pixels.")
print(f"🟢 Mosaic dimensions: {nx}×{ny} tiles ({img.shape[1]}×{img.shape[0]} pixels).")


# 2. Compute high-dimensional image descriptions
### Methods include an image pyramid or a neural network

In this section, each image is represented as a numerical feature vector that captures its visual characteristics. Two methods are available:

* Image Pyramid: Uses multi-scale Gaussian blurring to create a structured pixel-based representation.
* Neural Network (CNN): Uses a pretrained deep learning model (InceptionV3) to extract perceptual features.

These representations serve as input for dimensionality reduction techniques like UMAP or T-SNE in the next Section.

In [ ]:
# -----------------------------------
# FUNCTION TO MAKE BLURRED IMAGE SETS

# skimage.filters is imported here because 
# it's only used in this one place. 
from skimage.filters import gaussian

# Applies Gaussian blur to a batch of images.
def build_blurred(images, sigma):
    # Parameters:
    #    images (np.ndarray): Batch of images.
    #    sigma (float): Standard deviation for Gaussian kernel.
    # Returns:
    #    np.ndarray: Blurred images in uint8 format.
    blurred = []
    for image in images:
        # Apply Gaussian blur (output is normalized to [0, 1])
        blurred_image = gaussian(image, sigma=sigma, channel_axis=-1)
        
        # Scale back to [0, 255] and convert to uint8
        blurred_image = np.clip(blurred_image * 255, 0, 255).astype(np.uint8)
        blurred.append(blurred_image)

    return np.array(blurred)

# -----------------------------------------------
# GENERATE BLURRED IMAGE SETS
# We apply different levels of blur to each image in the dataset.
# The goal is to create a "multi-scale" representation for UMAP.
# Higher `sigma` values result in more aggressive blurring.

nx = int(img.shape[1] / TILE_SIZE)  # Number of columns
ny = int(img.shape[0] / TILE_SIZE)  # Number of rows
blur_levels = [4, 2, 1]    # Different blur intensities

# Apply Gaussian blur at different levels and visualize the results
blurred_0 = build_blurred(images, blur_levels[0])
imshow(make_mosaic(blurred_0, nx=nx, ny=ny), retina=True)  # Most blurred

blurred_1 = build_blurred(images, blur_levels[1])
imshow(make_mosaic(blurred_1, nx=nx, ny=ny), retina=True)  # Medium blur

blurred_2 = build_blurred(images, blur_levels[2])
imshow(make_mosaic(blurred_2, nx=nx, ny=ny), retina=True)  # Light blur

In [ ]:
# -----------------------------------
# CONSTRUCT THE IMAGE FEATURE PYRAMID

# Number of individual image patches
n = len(images)

# ⚡ Feature Stacking:
# We create a feature matrix (`pyr`) where each row represents an image,
# and each column contains pixel values from different levels of blurring.
# This allows UMAP (in the next step) to learn a representation 
# that takes multiple scales of detail into account.

pyr = np.hstack((
    blurred_0.reshape(n, -1),  # Strongly blurred images
    blurred_1.reshape(n, -1),  # Moderately blurred images
    blurred_2.reshape(n, -1),  # Lightly blurred images
    images.reshape(n, -1)      # Original images
))

# ✅ After this step:
# `pyr` is now a 2D array of shape (n, feature_dim), where:
#   - `n` is the number of image patches.
#   - `feature_dim` is the combined size of all image versions.
# This feature representation may be used as input to UMAP/T-SNE in later steps.


In [ ]:
# ---------------------------------------
# FEATURE EXTRACTION!
#
# Depending on the value of the FEATURE_METHOD setting (see first cell), 
# we choose the input data for dimensionality reduction. Options: 
# FEATURE_METHOD == "cnn" - Uses a neural network to describe each image with 2048 numbers.
# FEATURE_METHOD == "pyramid" - Uses an image pyramid instead; the pixels are the features. 
#
# NOTE: The neural net (cnn) option downloads a 104MB model, and can take some time.
# Uses InceptionV3 with Reduced Size (107x107 instead of 299x299) to avoid out-of-memory errors.
# See https://stackoverflow.com/questions/57421842/image-size-of-256x256-not-299x299-fed-into-inception-v3-model-pytorch-and-wo

if FEATURE_METHOD == "cnn":
    import torch
    import torchvision.models as models
    import torchvision.transforms as transforms

    # Select device (use GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load InceptionV3 model
    print("⏳ Loading InceptionV3 model with reduced input size (107x107)...")
    model = models.inception_v3(weights="IMAGENET1K_V1").to(device)
    model.fc = torch.nn.Identity()  # Remove final classification layer
    model.AuxLogits = None  # Disable auxiliary classifier
    model.eval()  # Set to evaluation mode
    print("✅ InceptionV3 model loaded.")

    # Preprocessing: Resize to 107x107 (instead of 299x299)
    preprocess = transforms.Compose([
        transforms.Resize((107, 107)),  # Reduce input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
    ])

    @torch.no_grad()  # Disable gradient tracking for efficiency
    def extract_features(images, batch_size=16):
        """
        Extract CNN features in batches to prevent out-of-memory errors.
        Uses a reduced input size (107x107) for efficiency.
        Parameters:
            images (np.ndarray): Image batch of shape (num_images, 32, 32, 3).
            batch_size (int): Number of images per batch.
        Returns:
            np.ndarray: Extracted feature vectors, shape (num_images, 2048).
        """
        features_list = []

        # Process images in mini-batches
        for i in range(0, len(images), batch_size):
            batch_images = images[i : i + batch_size]  # Slice a batch
            
            # Convert images to tensors
            images_resized = []
            for img in batch_images:
                img_pil = transforms.ToPILImage()(img)  # Convert NumPy to PIL Image
                img_tensor = preprocess(img_pil).unsqueeze(0).to(device)  # Preprocess + add batch dim
                images_resized.append(img_tensor)

            batch_tensor = torch.cat(images_resized, dim=0)  # Stack into a batch tensor
            
            # Extract features
            with torch.no_grad():
                batch_features = model(batch_tensor)
                batch_features = batch_features.view(batch_features.size(0), -1)  # Flatten

            # Move to CPU & store results
            features_list.append(batch_features.cpu().numpy())

        # Combine all batch results into a single array
        return np.vstack(features_list)

    print("⏳ Extracting CNN feature vectors...")
    if device == "cpu":
        print("Using cpu, this could take a bit.")
    source = extract_features(images, batch_size=16)  # Adjust batch size as needed
    print(f"✅ Extracted {source.shape[1]}D feature vectors for {source.shape[0]} images.")

else:
    # i.e. elif FEATURE_METHOD == "pyramid":
    print("⚠️ Using image pyramid instead of CNN features.")
    # Use the images' pixels themselves as their own feature representation. 
    # Uncomment one of the lines below to experiment with different sources:
    # source = images     # Use original images
    # source = blurred_0  # Use blurred images
    source = pyr





---
# 3. Dimensionality Reduction:
### Reducing high-dimensional image descriptions into a 2D "Embedding" with UMAP or T-SNE

High-dimensional image feature vectors are mapped into a 2D space using UMAP or T-SNE, techniques that preserve visual similarity. This step organizes images based on their features, clustering similar ones together. The resulting 2D space, called an "embedding" serves as a layout for arranging images in a meaningful way in the final visualization.

In [ ]:
# ---------------------------------------
# APPLY UMAP OR T-SNE FOR DIMENSIONALITY REDUCTION
# Reduces high-dimensional image data into a 2D space.
# Note: This cell can take some time — e.g. ~5s for 1K points

# ---------------------------------------
# 1️⃣ Import necessary libraries

import json
import os
from umap import UMAP
from sklearn.manifold import TSNE

# ---------------------------------------
# 2️⃣ CONFIGURE UMAP OR T-SNE PARAMETERS
# You can also add `random_state=12345` to set the UMAP/T-SNE random seed to 12345.
print(f"🔄 Running {REDUCTION_METHOD} for dimensionality reduction...")

if REDUCTION_METHOD == "UMAP":
    reducer = UMAP(
        n_neighbors=15,     # Larger values (e.g., 50) preserve more global structure
        min_dist=0.1,       # Controls how tightly points are packed
        n_components=2,     # Number of output dimensions (keep at 2D)
        metric="euclidean"  # Other options: "cosine", "manhattan", "correlation"
    )
    print(f"⚠️ Note: UMAP may trigger a FutureWarning; you can safely ignore this.")
    
elif REDUCTION_METHOD == "TSNE":
    reducer = TSNE(
        n_components=2,      # Number of output dimensions (keep at 2D)
        perplexity=30,       # Affects clustering (try values between 5 and 50)
        early_exaggeration=12,
        learning_rate=200,
        init="random"        # Can also use "pca"
    )
else:
    raise ValueError(f"❌ Invalid REDUCTION_METHOD: {REDUCTION_METHOD}. Choose 'UMAP' or 'TSNE'.")

# ---------------------------------------
# 3️⃣ RUN UMAP OR T-SNE & GENERATE EMBEDDING
# An "embedding" is a lower-dimensional representation of the data, 
# where similar images are placed closer together in the new space.
#
# ⚠️ Note: UMAP may trigger a FutureWarning related to `force_all_finite`
# being renamed to `ensure_all_finite` in a future version of scikit-learn.
# This warning is harmless and does NOT affect the results; you can safely ignore it.

%time Y = reducer.fit_transform(source.reshape(images.shape[0], -1).astype(np.float64))

# ---------------------------------------
# 4️⃣ Normalize Output to the range [0,1] for visualization
Y -= Y.min(axis=0)
Y /= Y.max(axis=0)

print(f"✅ {REDUCTION_METHOD} completed.")

# ---------------------------------------
# 5️⃣ SAVE EMBEDDING POSITIONS AS JSON
# Save the (x, y) 2D embedding coordinates for each image.
# This will generate a file with contents like:
# {
#  "image_001.png": {"x": 0.1234, "y": 0.5678},
#  "image_002.png": {"x": 0.2345, "y": 0.6789},
# }

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
embedding_positions = {
    filename: {"x": float(pos[0]), "y": float(pos[1])} 
    for filename, pos in zip(filenames, Y)
}
embedding_json_path = os.path.join(output_dir, f"{REDUCTION_METHOD}_positions.json")
with open(embedding_json_path, "w") as f:
    json.dump(embedding_positions, f, indent=2)
print(f"💾 Saved embedding positions to {embedding_json_path}")

In [ ]:
canvas = plot_images(images, Y, canvas_shape=(2048, 2048, 3), blend=np.minimum, fill=255)
imshow(canvas, retina=True)

---
# 4. Rectifying the 2D Embedding:
### Mapping T-SNE/UMAP Points to a Grid by Solving the Linear Assignment Problem

The 2D embedding from UMAP or T-SNE arranges images organically, but the points are not perfectly aligned to a grid. To correct this, we solve the Linear Assignment Problem (LAP) using an optimization algorithm. This step assigns each image to the nearest available grid position while preserving relative structure.

In [ ]:
# -----------------------------
# SOLVE THE ASSIGNMENT PROBLEM: 
# MAPPING UMAP POINTS TO A GRID

from lap import lapjv  # Linear Assignment Problem (LAP) solver
from scipy.spatial.distance import cdist  # Computes pairwise distances
import json
import os

# 1️⃣ Define the target grid dimensions
nx = int(img.shape[1] / TILE_SIZE) # Number of grid columns
ny = int(img.shape[0] / TILE_SIZE) # Number of grid rows

# 2️⃣ Generate a uniform 2D grid of (x, y) target positions
xv, yv = np.meshgrid(np.linspace(0, 1, nx), np.linspace(0, 1, ny))  # Evenly spaced grid
grid = np.dstack((xv, yv)).reshape(-1, 2)  # Convert to list of 2D points

# 3️⃣ Compute the pairwise squared Euclidean distance between UMAP points and grid points
#    - `Y` contains the 2D coordinates of images in the UMAP space.
#    - `grid` contains the target positions in a structured layout.
#    - `cdist()` calculates all pairwise distances.
%time cost = cdist(grid, Y, 'sqeuclidean')  # (num_grid_points, num_images)

# 4️⃣ Convert cost matrix to integer values (LAP solver works faster with integers)
cost = cost * (100000. / cost.max())  # Scale costs to large integers
cost = cost.astype(int)  # Convert to integer type

# 5️⃣ Solve the Linear Assignment Problem (LAP) using Jonker-Volgenant Algorithm
# See: https://en.wikipedia.org/wiki/Hungarian_algorithm. 
# Here we compute the optimal assignment, and print the execution time 
# Note: this implementation sorts black padding squares, technically incorrect. 
totalDataPoints = nx * ny  # Number of points to assign
%time min_cost, row_assigns, col_assigns = lapjv(cost, extend_cost=True)

# 6️⃣ Map embedding positions (`Y`) to the nearest grid positions (`grid_jv`)
grid_jv = grid[col_assigns[:totalDataPoints]]

# 7️⃣ Visualization: Draw arrows from embedding positions to assigned grid positions
plt.figure(figsize=(8, 8))
for start, end in zip(Y, grid_jv):
    plt.arrow(start[0], 1 - start[1],  # Flip Y-axis by subtracting from 1
              end[0] - start[0], -(end[1] - start[1]),  # Negate the Y difference
              head_length=0.01, head_width=0.01)
plt.show()


# -------------------------------
# 8️⃣ Save grid positions to JSON.
# Output file (grid_positions.json) looks like:
# {
#   "grid_size": {
#     "nx": 41,
#     "ny": 26
#   },
#   "positions": {
#     "kress_00001.png": {
#       "col": 26,
#       "row": 3
#     },
# etc.
      
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
grid_positions = {
    "grid_size": {"nx": nx, "ny": ny},  # Store metadata about the grid size
    "positions": {
        filename: {
            "col": round(pos[0] * (nx - 1)),  # Scale normalized value to grid
            "row": round(pos[1] * (ny - 1))   # Scale normalized value to grid
        }
        for filename, pos in zip(filenames, grid_jv)
    }
}
grid_json_path = os.path.join(output_dir, "grid_positions.json")
with open(grid_json_path, "w") as f:
    json.dump(grid_positions, f, indent=2)
print(f"💾 Saved grid positions to {grid_json_path}")

In [ ]:
# ------------------------------
# RECONSTRUCT THE ORDERED MOSAIC

# 1️⃣ Swap x and y coordinates to match the (row, column) ordering of the grid
grid_tuples = [(y, x) for (x, y) in map(tuple, grid_jv)]

# 2️⃣ Pair each image with its corresponding grid position
image_grid_pairs = list(zip(grid_tuples, images))

# 3️⃣ Sort the image-grid pairs by grid position (top-left to bottom-right)
sorted_pairs = sorted(image_grid_pairs)  # Sorts by (y, x) automatically

# 4️⃣ Extract the images from the sorted pairs (now correctly ordered for the mosaic).
# Note that the position data is now ignored.
sorted_images = [image for (position, image) in sorted_pairs]

# 5️⃣ Arrange sorted images into a final mosaic nparray
mosaic = make_mosaic(sorted_images, nx=nx, ny=ny)

# 6️⃣ Display the final, neatly arranged mosaic (TRUE SIZE)
# Note: `imshow(mosaic, retina=True)` is not pixel-perfect.
import PIL.Image
from IPython.display import display, HTML

# Save the mosaic as an image file
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
output_filename = "final_mosaic.png"
# Save the mosaic in the output directory
output_filename = os.path.join(output_dir, output_filename)
PIL.Image.fromarray(mosaic).save(output_filename)

# 7️⃣ (Force Jupyter to) display the image at its true size using HTML
# Otherwise Jupyter may resize the image and add a dumb border. 
display(HTML(f'<img src="{output_filename}" width="{mosaic.shape[1]}" height="{mosaic.shape[0]}" style="border:0px;">'))
print(f"✅ Generated UMAP mosaic with dimensions: {mosaic.shape[1]} x {mosaic.shape[0]} pixels")
